In [1]:
import pandas as pd
import numpy as np
import os
import utils

In [2]:
features_1 = [
    'sx_roll',
    'sx_pitch',
    'sx_yaw',
    'sx_thumb',
    'sx_x',
    'sx_y',
    'sx_z',
    'sx_forefinger',
    'sx_middle_finger',
    'sx_ring_finger',
    'sx_little_finger',
    'dx_roll',
    'dx_pitch',
    'dx_yaw',
    'dx_x',
    'dx_y',
    'dx_z',
    'dx_thumb',
    'dx_forefinger',
    'dx_middle_finger',
    'dx_ring_finger',
    'dx_little_finger'
    ]



In [3]:
#path = "./tctodd/"
path = "../../Desktop/MML Project/tctodd/"
dirs = os.listdir(path=path)
weeks = sorted([i for i in dirs if i != ".DS_Store"])
filenames = sorted(os.listdir(path+weeks[1]))

data = []
labels = dict()
label_cnt = 0

for w in weeks:
    temp_path = path+w+"/"
    filenames = sorted(os.listdir(temp_path))
    for fn in filenames:
        label = fn.split('.')[0][:-2]
        
        if label not in labels:
            labels[label] = label_cnt
            label_cnt += 1
            
        data.append({'label':labels[label], 'time_series':pd.read_csv(temp_path+fn, header=None, sep='\t',).values})
        

In [4]:
df = pd.DataFrame(data, columns=['label', 'time_series'])
print(df['label'].value_counts())
df.head()

93    27
68    27
50    27
52    27
54    27
      ..
35    27
37    27
39    27
41    27
0     27
Name: label, Length: 95, dtype: int64


,label,time_series
0,0,"[[-0.06490900000000001, 0.034318, -0.043963999..."
1,0,"[[-0.10705899999999999, -0.126109, -0.053742, ..."
2,0,"[[-0.061426999999999995, -0.082576, -0.1029910..."
3,1,"[[-0.128178, 0.02695, -0.050126, 0.455028, 0.4..."
4,1,"[[-0.143672, -0.14441600000000002, -0.047447, ..."


In [ ]:
##MISSING THE PARAMETER SEARCH?
import sklearn as sk
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from k_eros import *
import itertools
from tqdm import tqdm
seed = 0

X = df['time_series']
y = df['label']

# 10-fold cross validation


#best_combination = ('poly', 1e-05, 5, 23)
n_pcs = 55


##LINEAR KERNEL
params = [['linear'], [0.0001, 0.0005, 0.001, 0.005, 0.01, 1]]
params_comb = list(itertools.product(*params))
##RBF KERNEL
params = [['rbf'], [0.0001, 0.0005, 0.001, 0.005, 0.01, 1], [0.001, 0.01, 1, 2, 5, 10]]
params_comb += list(itertools.product(*params))
##POLYNOMIAL KERNEL
params = [['poly'], [0.0001, 0.0005, 0.001, 0.005, 0.01, 1], [0.001, 0.01, 1, 2, 5, 10], [3, 6, 10, 15, 20, 25, 30], [0, 0.1, 0.5, 1, 5, 10]]
params_comb += list(itertools.product(*params))
best_comb = ''
best_acc = 0
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    stratify=y, 
                                                    test_size=0.1, seed=seed)

for params in tqdm(params_comb, desc='doing parameters search...'):
    X_train_matrix = np.vstack(X_train)
    means_train = np.mean(X_train_matrix, axis=0)
    vars_train = np.var(X_train_matrix, axis=0)

    S, v_list_train = compute_S_matrix(X_train, means_train, vars_train)
    _, v_list_test = compute_S_matrix(X_test, means_train, vars_train)
    w = compute_weight_vector(S, algorithm=2)
    K_eros_train_mc, V, eig_vals = perform_PCA(len(X_train), weight_vector=w, v_list=v_list_train)
    Y, K_eros_test_mc = project_test_data(len(X_train), len(X_test), w, v_list_train, v_list_test, K_eros_train_mc, V)

    if len(params) == 2:
        combination = tuple([params[0], params[1], 1, 3, 0])
    elif len(params) == 3:
        combination = tuple([params[0], params[1], params[2], 3, 0])
    else:
        combination = params
    svc = SVC(kernel=combination[0], C=combination[1], gamma=combination[2], degree=combination[3], coef0=combination[4])
    princ_components = V[:, n_pcs]
    svc.fit(princ_components, y_train.values)
    test_princ_components = Y[:, :n_pcs]
    predictions = svc.predict(test_princ_components)
    res = accuracy_score(y_test.values, predictions)
if res > best_acc:
    best_acc = res
    best_comb = combination
    print(f'Found best combination! {best_comb} w. accuracy of {best_acc}.')

In [ ]:
#for the best params combination il valore dell accuracy per 10Kfold
skf = sk.model_selection.StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
mean_accuracy = 0
n_princ_cs = [1, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55]
res_per_pc = np.zeros(len(n_princ_cs))
for train_index, test_index in skf.split(X, y):
    X_train_matrix = np.vstack(X_train)
    means_train = np.mean(X_train_matrix, axis=0)
    vars_train = np.var(X_train_matrix, axis=0)

    S, v_list_train = compute_S_matrix(X_train, means_train, vars_train)
    _, v_list_test = compute_S_matrix(X_test, means_train, vars_train)
    w = compute_weight_vector(S, algorithm=2)
    K_eros_train_mc, V, eig_vals = perform_PCA(len(X_train), weight_vector=w, v_list=v_list_train)
    Y, K_eros_test_mc = project_test_data(len(X_train), len(X_test), w, v_list_train, v_list_test, K_eros_train_mc, V)
    svc = SVC(kernel=best_comb[0], C=best_comb[1], gamma=best_comb[2], degree=best_comb[3], coef0=best_comb[4])
    for i, n_pc in enumerate(n_princ_cs):
        princ_components = V[:, :n_pc]
        svc.fit(princ_components, y_train.values)
        test_princ_components = Y[:, :n_pc]
        predictions = svc.predict(test_princ_components)
        res = accuracy_score(y_test.values, predictions)
        res_per_pc[i] += res
print(f'Reached an accuracy of {res_per_pc[-1]/10}.')
svc_res_per_pc = res_per_pc/10


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import ParameterGrid
criterion = ['gini', 'entropy']
n_estimators = [10, 25, 50, 100, 200]
max_depth = [10, 20, 30, 50, 100]
min_samples_split = [2, 6, 10] # minimum sample number to split a node
min_samples_leaf = [1, 3, 4] # minimum sample number that can be stored in a leaf node
params_grid = {'n_estimators': n_estimators,
'max_depth': max_depth,
'min_samples_split': min_samples_split,
'min_samples_leaf': min_samples_leaf,
}
X_train, X_test, y_train, y_test = train_test_split(X, y,stratify=y, test_size=0.1, seed=seed)
best_comb = ''
best_acc = 0
for params in list(ParameterGrid(params_grid)):
    X_train_matrix = np.vstack(X_train)
    means_train = np.mean(X_train_matrix, axis=0)
    vars_train = np.var(X_train_matrix, axis=0)

    S, v_list_train = compute_S_matrix(X_train, means_train, vars_train)
    _, v_list_test = compute_S_matrix(X_test, means_train, vars_train)
    w = compute_weight_vector(S, algorithm=2)
    K_eros_train_mc, V, eig_vals = perform_PCA(len(X_train), weight_vector=w, v_list=v_list_train)
    Y, K_eros_test_mc = project_test_data(len(X_train), len(X_test), w, v_list_train, v_list_test, K_eros_train_mc, V)
    rf = RandomForestClassifier(*params)
    princ_components = V[:, :n_pcs]
    rf.fit(princ_components, y_train.values)
    test_princ_components = Y[:, :n_pcs]
    predictions = rf.predict(test_princ_components)
    res = accuracy_score(y_test.values, predictions)
if res > best_acc:
    best_acc = res
    best_comb = params
    print(f'Found best combination! {best_comb} w. accuracy of {best_acc}.')
    



In [ ]:
#for the best params combination il valore dell accuracy per 10Kfold
skf = sk.model_selection.StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
n_princ_cs = [1, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55]
res_per_pc = np.zeros(len(n_princ_cs))
for train_index, test_index in skf.split(X, y):
    X_train = X[train_index]
    y_train = y[train_index]
    X_test = X[test_index]
    y_test = y[test_index]
    X_train_matrix = np.vstack(X_train)
    means_train = np.mean(X_train_matrix, axis=0)
    vars_train = np.var(X_train_matrix, axis=0)

    S, v_list_train = compute_S_matrix(X_train, means_train, vars_train)
    _, v_list_test = compute_S_matrix(X_test, means_train, vars_train)
    w = compute_weight_vector(S, algorithm=2)
    K_eros_train_mc, V, eig_vals = perform_PCA(len(X_train), weight_vector=w, v_list=v_list_train)
    Y, K_eros_test_mc = project_test_data(len(X_train), len(X_test), w, v_list_train, v_list_test, K_eros_train_mc, V)
    rf = RandomForestClassifier(*best_comb)
    for i, n_pc in enumerate(n_princ_cs):
        princ_components = V[:, :n_pc]
        rf.fit(princ_components, y_train.values)
        test_princ_components = Y[:, :n_pc]
        predictions = rf.predict(test_princ_components)
        res = accuracy_score(y_test.values, predictions)
        res_per_pc[i] += res

print(f'Reached an accuracy of {res_per_pc[-1]/10}.')
rf_res_per_pc = res_per_pc/10


In [ ]:
print('SVC accuracy x n principal components: ', svc_res_per_pc)
print('RF accuracy x n principal components: ', rf_res_per_pc)